In [4]:
import dataclasses
import pandas as pd
# 表示量
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

# データを読み込む
tag_skill_df = pd.read_csv("./Notion/Tags×スキル対応表.csv")
skill_df = pd.read_csv("./iCD2022/スキル一覧（スキル項目）.csv")
knowledge_df = pd.read_csv("./iCD2022/スキル一覧（知識項目）.csv")
knowledge_df = knowledge_df.drop("スキル項目識別子", axis=1)
career_skill_df = pd.read_csv("./iCD2022/職種×スキル対応表.csv")
career_df = pd.read_csv("./iCD2022/職種一覧.csv")


### 1. Tags×スキル対応表の作成
Tagsとスキルの対応関係の表を手動で作成する。
1. スキル一覧（スキル項目）を眺めて対応しそうなスキルを見つける。
2. スキル一覧（知識項目）を眺めてさらに対応しそうな知識項目を見つける。
3. Tags×スキル対応表に書き出す。

<img src='https://private-user-images.githubusercontent.com/128585616/254515297-c0df7ed4-9f5d-4a75-9d12-1737088c0e0b.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTEiLCJleHAiOjE2ODk3NTYxODMsIm5iZiI6MTY4OTc1NTg4MywicGF0aCI6Ii8xMjg1ODU2MTYvMjU0NTE1Mjk3LWMwZGY3ZWQ0LTlmNWQtNGE3NS05ZDEyLTE3MzcwODhjMGUwYi5wbmc_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTYmWC1BbXotQ3JlZGVudGlhbD1BS0lBSVdOSllBWDRDU1ZFSDUzQSUyRjIwMjMwNzE5JTJGdXMtZWFzdC0xJTJGczMlMkZhd3M0X3JlcXVlc3QmWC1BbXotRGF0ZT0yMDIzMDcxOVQwODM4MDNaJlgtQW16LUV4cGlyZXM9MzAwJlgtQW16LVNpZ25hdHVyZT05ZGU1M2E2ODExZDE2MzNkNTI0ZjBmN2M2NDU0MTI4ZDE1YTllZTM0ZmQxNzE2OWYyOTYyYmY2NGQ0N2QyMmRhJlgtQW16LVNpZ25lZEhlYWRlcnM9aG9zdCZhY3Rvcl9pZD0wJmtleV9pZD0wJnJlcG9faWQ9MCJ9.cG9M64l2vl0WAtm5YshtHjA_7aNWcxO2QilCODMl4tk' alt="スキルディクショナリ構成図" />

#### 1.1. スキル一覧（スキル項目）を眺めて対応しそうなスキルを見つける。

In [ ]:
@dataclasses.dataclass(frozen=True)
class SkillCategory:
    all: str = '.*'
    methodology: str = 'メソドロジ'
    technology: str = 'テクノロジ'
    human: str = 'ITヒューマンスキル'
    related: str = '関連知識'

@dataclasses.dataclass(frozen=True)
class SkillClass:
    all: str = '.*'
    # メソドロジ
    strategy: str = '.*戦略.*'                     # Skill Code: S11      
    planning: str = '.*企画.*'                     # Skill Code: S12
    implement: str = '.*実装.*'                    # Skill Code: S13
    utilization: str = '.*利活用.*'                # Skill Code: S14
    support: str = '.*支援活動.*'                  # Skill Code: S15
    # テクノロジ
    system: str = '.*（システム）.*'               # Skill Code: S21
    development: str = '.*（開発）.*'              # Skill Code: S22
    operation: str = '.*保守・運用.*'              # Skill Code: S23
    non_function: str = '.*非機能要件.*'           # Skill Code: S24
    embedded: str = '.*組込み・計測・制御.*'       # Skill Code: S25
    common: str = '.*共通技術.*'                  # Skill Code: S26
    # ITヒューマンスキル
    creativity: str = '.*創造力.*'                    # Skill Code: S41001
    action: str = '.*実行・実践力.*'                  # Skill Code: S41002
    communication: str = '.*コミュニケーション力.*'    # Skill Code: S41003
    # 関連知識
    business: str = '.*ビジネスインダストリ.*'           # Skill Code: S31001
    corporate: str = '.*企業活動.*'                     # Skill Code: S31002
    standard: str = '.*法規・基準・標準.*'               # Skill Code: S31003
    
# スキルカテゴリを絞る
# SkillCategory. 以下の部分を補完機能を使用して入力する
# select_df = skill_df.query("スキルカテゴリ.str.match('{0}')".format(SkillCategory.all))

# スキル分類を絞る
# SkillClass. 以下の部分を補完機能を使用して入力する
select_df = skill_df.query("スキル分類.str.match('{0}')".format(SkillClass.implement))

# 表示する
select_df.drop(["スキルカテゴリ識別子", "スキル分類コード", "スキル分類識別子", "スキル項目識別子"], axis=1).head(500)

#### 1.2. スキル一覧（知識項目）を眺めてさらに対応しそうな知識項目を見つける。

In [6]:
# データの結合
skill_know_df = pd.merge(skill_df, knowledge_df, on=["スキル項目コード"], how="inner")

# １で絞ったスキル項目に対応するスキル項目コードを入力し、知識項目を見つける
# 正規表現を使用可能。例：下二けた任意"S1100100.."
# skill_item_code = "S130100070"
# select_knowledge = skill_know_df.query("スキル項目コード.str.match('{0}')".format(skill_item_code))

# または、知識項目で検索する
search_word = ".*アジャイル.*"
select_knowledge = skill_know_df.query("知識項目.str.match('{0}')".format(search_word))

# 表示する
select_knowledge[["スキル項目コード", "知識項目コード", "スキル分類", "知識項目"]].head(500)

,スキル項目コード,知識項目コード,スキル分類,知識項目
1444,S130020030,K008,（実装） ソフトウェアエンジニアリング手法,アジャイル開発
1560,S130020070,K019,（実装） ソフトウェアエンジニアリング手法,アジャイル・ソフトウェア開発
1561,S130020070,K020,（実装） ソフトウェアエンジニアリング手法,アジャイルプロセスモデル
1562,S130020070,K021,（実装） ソフトウェアエンジニアリング手法,アジャイル法（RAD、XP、Scrum、FDD）
1627,S130020070,K086,（実装） ソフトウェアエンジニアリング手法,ライフサイクルモデル（アジャイル、ヘビーウェイト、ウォーターフォール、スパイラルなど）
4769,S210020040,K088,（システム） ソフトウェアの構築技術,開発方式（例:アジャイル、RAD、ラピッドプロトタイピングなど）
6975,S220020010,K002,（開発） システム開発管理技術,アジャイル開発


#### 1.3. Tags×スキル対応表に書き出す。
上記２で表示した、スキル項目コードと知識項目コードの中でTagsと適合するものを下記CSVファイルに追加する。
- <a href="./Notion/Tags×スキル対応表.csv">Tags×スキル対応表</a>

### 2. なりたい人材像を特定する

In [ ]:
career_df

# 解説が欠けている場合、下記をコメントアウトして表示できる
# career_code = "HU-040-010"
# select_career_df = career_df.query("専門分野コード.str.match('{0}')".format(career_code))
# print(select_career_df.drop(["解説"], axis=1).iloc[0], "\n\n解説")
# print(select_career_df["解説"].iloc[0])

### 3. 習得したスキルと人材像の表示

#### 3.0. 準備

In [ ]:
# 2.で特定した専門分野コードを指定する
career_code = "HI-060-010"

# テーブルの結合
skill_know_df = pd.merge(skill_df, knowledge_df, on=["スキル項目コード"], how="inner")
career_all_skill_df = pd.merge(career_df, career_skill_df, on=["専門分野コード"], how="inner")

pbl_skill_df = pd.merge(skill_know_df, tag_skill_df, on=["スキル項目コード", "知識項目コード"], how="right")
pbl_career_df = pd.merge(career_all_skill_df, pbl_skill_df, on=["スキル項目コード"], how="inner")
pbl_career_df = pbl_career_df.drop(["出自", "解説", "スキルカテゴリ識別子", "スキル分類コード", "スキル分類識別子", "スキル項目識別子", "知識項目識別子"], axis=1)

# PBL活動で学んだスキルが対応する人材像を表示する
pbl_career_df

#### 3.1. なりたい人材像に対して満たしているスキルを表示する

In [32]:
career_code = "HI-040-010"
select_pbl_career_df = pbl_career_df.query("専門分野コード.str.match('{0}')".format(career_code))
# 満たしているスキルのコードを取得する
meet_skill_codes = select_pbl_career_df["スキル項目コード"].to_list()
meet_knowlege_codes = select_pbl_career_df["知識項目コード"].to_list()

select_pbl_career_df

,専門分野コード,人材類型,人材像,職種,専門分野,スキル項目コード,スキルカテゴリ,スキル分類,スキル項目,知識項目コード,知識項目,TAG_ID


#### 3.2. なりたい人材像に対して足らないスキルを表示

In [33]:
select_career_all_skill_df = career_all_skill_df.query("専門分野コード.str.match('{0}')".format(career_code))
select_career_all_skill_df = pd.merge(select_career_all_skill_df, skill_know_df, on=["スキル項目コード"], how="left")

# 満たしているスキルを除外する
for meet_skill_code, meet_knowlege_code in zip(meet_skill_codes, meet_knowlege_codes):
    select_career_all_skill_df = select_career_all_skill_df.query("スキル項目コード != '{0}' | 知識項目コード != '{1}'".format(meet_skill_code, meet_knowlege_code))
    
select_career_all_skill_df.drop(["出自", "解説", "スキルカテゴリ識別子", "スキル分類コード", "スキル分類識別子", "スキル項目識別子", "知識項目識別子"], axis=1)


,専門分野コード,人材類型,人材像,職種,専門分野,スキル項目コード,スキルカテゴリ,スキル分類,スキル項目,知識項目コード,知識項目
0,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S110060010,メソドロジ,（戦略） システム戦略立案手法,システム化戦略手法,K001,ANSI
1,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S110060010,メソドロジ,（戦略） システム戦略立案手法,システム化戦略手法,K002,CRUD分析
2,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S110060010,メソドロジ,（戦略） システム戦略立案手法,システム化戦略手法,K003,CSF（Critical Success Factor）
3,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S110060010,メソドロジ,（戦略） システム戦略立案手法,システム化戦略手法,K004,EA（エンタープライズアーキテクチャ）
4,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S110060010,メソドロジ,（戦略） システム戦略立案手法,システム化戦略手法,K005,ER図
...,...,...,...,...,...,...,...,...,...,...,...
2136,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S310030110,関連知識,法規・基準・標準,標準化関連（ソフトウェア）,K004,ISO/IEC 12119-1994
2137,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S310030110,関連知識,法規・基準・標準,標準化関連（ソフトウェア）,K005,ISO/IEC/IEEE 29148
2138,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S310030110,関連知識,法規・基準・標準,標準化関連（ソフトウェア）,K006,ソフトウェア管理ガイドライン
2139,HI-040-010,ソリューション系,システムアーキテクト,ITアーキテクト,-,S310030120,関連知識,法規・基準・標準,標準化関連（プロジェクトマネジメント）,K001,ISO 10006（プロジェクトマネジメント）
